<a href="https://colab.research.google.com/github/sbh69840/SOT/blob/main/Object_tracking_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object tracking using OpenCV 
A simple object tracker built with OpenCV and JavaScript (for selecting the bounding box) \\
This tutorial is for datasets from VOT which consist of images instead of video.



In [1]:
# To execute some of the system commands like unzip and cp
import os
# Image processing library which contains trackers and image manipultation tools
import cv2
# Disply widgets to show output on the cell 
from IPython.display import HTML, Image
# Communicate between JS and Python
from google.colab.output import eval_js
# To display images because cv2.imshow doesn't work as it opens a new X-terminal
# which is not possible because a jupyter notebook instance doesn't allow that
from google.colab.patches import cv2_imshow
# To convert images to GIF
from PIL import Image
# high-level mathematical functions library
import numpy as np

In [2]:
# Clone the repository containing the dataset collected for the purpose of object tracking
!git clone https://github.com/sbh69840/SOT-dataset.git

Cloning into 'SOT-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 8), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [3]:
# Choose from the available zip files containing images of each frame named track{id}
track_video = "track1"

In [4]:
# Unzip the files from the zip file
os.system("unzip SOT-dataset/{}.zip".format(track_video))

0

In [5]:
# Resource : https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/

# List of available trackers in openCV
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
# Function used to select tracker
def select_tracker(tracker_id):
  # Make tracker and tracker_type global to access it outside the function
  global tracker
  global tracker_type
  tracker_type = tracker_types[tracker_id]
  # Set up tracker.
  if tracker_type == 'BOOSTING':
      tracker = cv2.TrackerBoosting_create()
  if tracker_type == 'MIL':
      tracker = cv2.TrackerMIL_create()
  if tracker_type == 'KCF':
      tracker = cv2.TrackerKCF_create()
  if tracker_type == 'TLD':
      tracker = cv2.TrackerTLD_create()
  if tracker_type == 'MEDIANFLOW':
      tracker = cv2.TrackerMedianFlow_create()
  if tracker_type == 'GOTURN':
      tracker = cv2.TrackerGOTURN_create()
  if tracker_type == 'MOSSE':
      tracker = cv2.TrackerMOSSE_create()
  if tracker_type == "CSRT":
      tracker = cv2.TrackerCSRT_create()

In [6]:
# Call the function to select the tracker
select_tracker(0)
# Print to check the name of the tracker
print(tracker)

<TrackerBoosting 0x7fab6bbece30>


In [7]:
# The first frame of the file
frame_name = "1.jpg"
frame_number = 1
# All the files have 12 digits in them so the rest of the spots are filled with 
# zeros. To see the file structure check the colab files section.
frame_name = frame_name.rjust(12,"0")
# Copy the first frame to the location where we can access the image from inside
# of HTML because the root of the localhost is located in the folder 
# /usr/local/share/jupyter
os.system("cp {0}/{1} /usr/local/share/jupyter/nbextensions/{1}".format(track_video,frame_name))

0

In [8]:
html_canvas = """
<!--
Create the canvas to display the image to mark the bounding box. 
-->
<canvas id="canvas"></canvas>
<!--
Button used to send the bounding box coordinates  
-->
<button id="capture">Capture</button>
<script>
<!--
//Canvas to draw the bounding box on 
-->
var canvas = document.getElementById('canvas');
<!--
//Width of the image 
-->
var width = %d;
<!--
//Height of the image 
-->
var height = %d;
<!--
//Set canvas width to the width of the image 
-->
canvas.style.width = width+'px';
<!--
//Set canvas height to the height of the image 
-->
canvas.style.height = height+'px';
<!--
//Get context of the canvas used to draw on the images
-->
var ctx = canvas.getContext('2d');
<!--
//Set context width and height according to image width and height
-->
ctx.canvas.width = width;
ctx.canvas.height = height;
<!--
//Load the image onto a image class to draw it on a canvas
-->
var base_image = new Image();
base_image.src = '%s';
base_image.width = width+'px';
base_image.height = height+'px';
base_image.onload = function(){
  ctx.drawImage(base_image,0,0,width,height);
}
<!--
//Call the function that keeps track of the mouse events and draws bounding boxes
-->
initDraw(canvas,ctx);
<!--
//Global variables used to send the coordinates that will be set in inside the function 
//initDraw(canvas,ctx)
-->
var x,y,width,height;
function initDraw(canvas,ctx) {
    <!--
    //function used to set the mouse position according to current position
    -->
    function setMousePosition(e) {
        var ev = e || window.event; //Moz || IE
        if (ev.pageX) { //Moz
            mouse.x = ev.pageX + window.pageXOffset;
            mouse.y = ev.pageY + window.pageYOffset;
        } else if (ev.clientX) { //IE
            mouse.x = ev.clientX + document.body.scrollLeft;
            mouse.y = ev.clientY + document.body.scrollTop;
        }
    };
    <!--
    //Dictionary used to store current x, current y position when the mouse moves
    //and also startX and startY to store the left corner point on first click
    -->
    var mouse = {
        x: 0,
        y: 0,
        startX: 0,
        startY: 0
    };
    <!--
    //local left, top, width and height used to keep track of bounding box inside
    //of this function
    -->
    var left=null,top=null,width=0,height=0;
    <!--
    //Function called when the mouse pointer is moved
    -->
    canvas.onmousemove = function (e) {
        <!--
        //set current position to the position of the mouse pointer on screen
        -->
        setMousePosition(e);
        <!--
        //check if left is null, if it is then there is no rectangle on the screen
        -->
        if (left !== null) {
            <!--
            //This will clear the entire canvas whenever there is already a rectangle 
            //on the screen because we only need one rectangle on the screen at 
            //a given time
            -->
            ctx.clearRect(0,0,canvas.width,canvas.height);
            <!--
            //After clearing the whole canvas the image will no longer be there
            //so draw it again
            -->
            ctx.drawImage(base_image,0,0,canvas.width,canvas.height);
            <!--
            //Set width, height, top, left values with respect to the startX
            //and startY positions
            -->
            width = Math.abs(mouse.x - mouse.startX);
            height = Math.abs(mouse.y - mouse.startY);
            left = (mouse.x - mouse.startX < 0) ? mouse.x : mouse.startX;
            top = (mouse.y - mouse.startY < 0) ? mouse.y : mouse.startY;
            <!--
            //Init function before drawing on canvas
            -->
            ctx.beginPath();
            <!--
            //Set the width of the line, strokecolor as per the user input
            -->
            ctx.lineWidth = "%d";
            ctx.strokeStyle = "%s";
            ctx.strokeRect(left, top, width, height);
            <!--
            //console.log(left,top,width,height);
            -->
            ctx.stroke();
        }
    }
    <!--
    // Invoke onclick function when clicked on canvas 
    -->
    canvas.onclick = function (e) {
      <!--
      // if left is null that means the rectangle is not there and you create
      // a new one by assigning startX and startY else assign coordinate values
      // of the rectangle according to current position of the mouse pointer
      -->
        if (left !== null) {
            window.x = left;
            window.y = top;
            window.width = width;
            window.height = height;
            left = null;
            canvas.style.cursor = "default";
            console.log("finsihed.");
        } else {
            console.log("begun.");
            mouse.startX = mouse.x;
            mouse.startY = mouse.y;
            left = mouse.x;
            top = mouse.y;
            canvas.style.cursor = "crosshair";
        }
    }
    
    
}
<!--
//Get the button from the document and send the coordinates to python using
//promise when the button is clicked
-->
var button = document.getElementById('capture');
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve([x,y,width,height]);
  }
});
</script>
"""

# Fuction used to create the html page in the cell and receive whatever data will
# be sent from the variable data in the javascript code above
# Return: tuple of (x, y, width, height) bounding box
def selectROI(filename='1.jpg', w=400, h=200, line_width=3,color="yellow"):
  display(HTML(html_canvas%(w,h,'/nbextensions/{}'.format(filename),line_width,color)))
  data = eval_js("data")
  return tuple(data)

In [9]:
# read the image to find the width and height 
frame = cv2.imread("{}/".format(track_video)+frame_name)
print(frame.shape)
# get the bounding box
bbox = selectROI(frame_name,frame.shape[1],frame.shape[0],3,"yellow")

(810, 480, 3)


MessageError: ignored

In [ ]:
bbox

(171, 385, 56, 71)

In [ ]:
# Initialize the tracker with the first frame
ok = tracker.init(frame,bbox)

In [ ]:
print(ok)

True


In [ ]:
while True:
  # clear_output()
  frame_number+=1
  frame_name1 = "{}.jpg".format(frame_number)
  frame_name1 = frame_name1.rjust(12,"0")
  frame1 = cv2.imread("{}/".format(track_video)+frame_name1)
  try:
    frame1.size
    print(frame_name1)
  except:
    break
  # Start timer
  timer = cv2.getTickCount()
  # Update the tracker
  ok, bbox = tracker.update(frame1)

  # Calculate Frames Per Second (FPS) 
  fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

  if ok:
    # Tracking success
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame1, p1, p2, (255,255,0), 3, 1)
  else:
    # Tracking failure
    cv2.putText(frame1, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
  
  # Display tracker type on frame
  cv2.putText(frame1, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0),2)
  
  # Display FPS on frame
  cv2.putText(frame1, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
  gif.append(frame1)
  # cv2_imshow(frame)


00000002.jpg
00000003.jpg
00000004.jpg
00000005.jpg
00000006.jpg
00000007.jpg
00000008.jpg
00000009.jpg
00000010.jpg
00000011.jpg
00000012.jpg
00000013.jpg
00000014.jpg
00000015.jpg
00000016.jpg
00000017.jpg
00000018.jpg
00000019.jpg
00000020.jpg
00000021.jpg
00000022.jpg
00000023.jpg
00000024.jpg
00000025.jpg
00000026.jpg
00000027.jpg
00000028.jpg
00000029.jpg
00000030.jpg
00000031.jpg
00000032.jpg
00000033.jpg
00000034.jpg
00000035.jpg
00000036.jpg
00000037.jpg
00000038.jpg
00000039.jpg
00000040.jpg
00000041.jpg
00000042.jpg
00000043.jpg
00000044.jpg
00000045.jpg
00000046.jpg
00000047.jpg
00000048.jpg
00000049.jpg
00000050.jpg
00000051.jpg
00000052.jpg
00000053.jpg
00000054.jpg
00000055.jpg
00000056.jpg
00000057.jpg
00000058.jpg
00000059.jpg
00000060.jpg
00000061.jpg
00000062.jpg
00000063.jpg
00000064.jpg
00000065.jpg
00000066.jpg
00000067.jpg
00000068.jpg
00000069.jpg
00000070.jpg
00000071.jpg
00000072.jpg
00000073.jpg
00000074.jpg
00000075.jpg
00000076.jpg
00000077.jpg
00000078.jpg

In [ ]:
print(type(gif[0]))
frame_number

<class 'numpy.ndarray'>


172

In [ ]:
# Resource: https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
import random
random.seed(10)
gifs = []
images = []# Images, just convert it into PIL.Image obj
n = len(gif)
ind = set(random.sample(list(range(0,n)),12))
cur_image_ind = 0
cur_ind = 0
imgs = [[] for _ in range(3)]
# Go through each image and convert them to Image object from numpy array
for image in gif:
  gifs.append(Image.fromarray(image.astype(np.uint8)))
  if cur_image_ind in ind:
    imgs[cur_ind%3].append(image_resize(image,200))
    cur_ind+=1
  cur_image_ind+=1
# Convert all the elements of gifs of type Image to a gif
gifs[0].save('temp_result.gif', save_all=True,optimize=True, append_images=gifs[1:], loop=0)

In [ ]:
for i in range(3):
  imgs[i] = np.hstack(imgs[i])
v_stack = np.vstack(imgs)


In [ ]:
cv2_imshow(v_stack)

In [ ]:
# Copy the gif to access it from HTML
os.system("cp temp_result.gif /usr/local/share/jupyter/nbextensions/temp_result.gif")

0

In [ ]:
# Display the gif, it will take some time because the GIF is not optimized and
# is heavy in terms of size
display_html = """
<img src="/nbextensions/temp_result.gif">
"""
display(HTML(display_html))